# **Modelo LLM Gemini avançado**

In [ ]:
import re
import spacy
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer

import warnings
warnings.filterwarnings('ignore')

In [ ]:
# Baixar recursos do NLTK
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

In [ ]:
# Carregar modelo do SpaCy para lematização
nlp = spacy.load("en_core_web_sm")

In [ ]:
# Base dados
df = pd.read_csv("/content/financial_phrase_bank_pt_br.csv")
df

In [ ]:
# Visualizando 5 primeiros dados
df.head()

In [ ]:
# Visualizando 5 últimos dados
df.info()

In [ ]:
# Tipo dados
df.dtypes

In [ ]:
# Linhas e colunas
df.shape

In [ ]:
# Agora, vamos renomear as colunas
df = df.rename(columns={"y": "Sentimento",
                        "text": "Text",
                        "text_pt": "Text_pt"})

# Visualizar o DataFrame com as colunas renomeadas
df.head()

# **Pré-processamento**

In [ ]:
# Função para limpar o texto

def limpar_texto(texto):
    # Converter para minúsculas
    texto = texto.lower()
    # Remover caracteres especiais, números e pontuações
    texto = re.sub(r'[^a-zA-Z\s]', '', texto)
    # Remover espaços em branco extras
    texto = re.sub(r'\s+', ' ', texto).strip()
    return texto

# Função para remover stop words
def remover_stop_words(texto):
    stop_words = set(stopwords.words('english'))
    palavras = word_tokenize(texto)
    texto_sem_stopwords = [word for word in palavras if word not in stop_words]
    return ' '.join(texto_sem_stopwords)

# Função para lematizar o texto
def lematizar_texto(texto):
    doc = nlp(texto)
    texto_lematizado = " ".join([token.lemma_ for token in doc])
    return texto_lematizado

# Aplicar as funções de pré-processamento à coluna "Text" e criar a coluna "Texto_limpo"
df['Texto_limpo'] = df['Text'].apply(limpar_texto)
df['Texto_limpo'] = df['Texto_limpo'].apply(remover_stop_words)
df['Texto_limpo'] = df['Texto_limpo'].apply(lematizar_texto)

In [ ]:
# Visualiznado
df.head()

In [ ]:
# Visualizando nova coluna
df.Texto_limpo.head()

# **Análise dados**

In [ ]:
sns.countplot(x='Sentimento', data=df)
plt.title('Distribuição de Sentimentos')
plt.xlabel('Sentimento')
plt.ylabel('Quantidade')
plt.show()

In [ ]:
from wordcloud import WordCloud

# Juntar todas as palavras em uma única string
texto_completo = ' '.join(df['Text'])

# Criar a nuvem de palavras
wordcloud = WordCloud(width=800, height=400, background_color ='white').generate(texto_completo)

# Plotar a nuvem de palavras
plt.figure(figsize=(10, 5))
plt.imshow(wordcloud, interpolation='bilinear')
plt.title('Nuvem de Palavras - Texto geral')
plt.axis('off')
plt.show()

In [ ]:
# Juntar todas as palavras em uma única string
texto_completo = ' '.join(df['Texto_limpo'])

# Criar a nuvem de palavras
wordcloud = WordCloud(width=800, height=400, background_color ='white').generate(texto_completo)

# Plotar a nuvem de palavras
plt.figure(figsize=(10, 5))
plt.imshow(wordcloud, interpolation='bilinear')
plt.title('Nuvem de Palavras - Texto limpo')
plt.axis('off')
plt.show()

In [ ]:
# Filtrar o DataFrame por cada sentimento
sentimentos_unicos = df['Sentimento'].unique()

# Criar uma nuvem de palavras para cada sentimento
for sentimento in sentimentos_unicos:

    # Filtrar o texto limpo para o sentimento atual
    texto_completo = ' '.join(df[df['Sentimento'] == sentimento]['Texto_limpo'])

    # Criar a nuvem de palavras
    wordcloud = WordCloud(width=800, height=400, background_color ='white').generate(texto_completo)

    # Plotar a nuvem de palavras
    plt.figure(figsize=(10, 5))
    plt.imshow(wordcloud, interpolation='bilinear')
    plt.title(sentimento)
    plt.axis('off')
    plt.show()

In [ ]:
# Tokenização texto

# Função para tokenizar o texto
def tokenizar_texto(texto):
    tokens = word_tokenize(texto)
    return tokens

# Aplicar a função de tokenização à coluna "Text"
df['text_Tokens'] = df['Text'].apply(tokenizar_texto)

In [ ]:
# Visualizando os tokens
df.text_Tokens

In [ ]:
# Contar a frequência de cada token
tokens_freq = pd.Series([token for sublist in df['text_Tokens'] for token in sublist]).value_counts()

# Plotar o gráfico de barras com seaborn
plt.figure(figsize=(10, 5))
sns.barplot(x=tokens_freq.index[:20], y=tokens_freq.values[:20])
plt.title('Top 20 Tokens')
plt.xlabel('Token')
plt.ylabel('Frequência')
plt.xticks(rotation=45, ha='right')
plt.show()

In [ ]:
# Função n_grams

# Importando biblioteca
from nltk.tokenize import word_tokenize
from nltk.util import ngrams
from collections import Counter

# Função para criar n-grams
def criar_ngrams(tokens, n):
    ngrams_list = list(ngrams(tokens, n))
    return ngrams_list

# Aplicar a função de criação de n-grams à coluna "Tokens"
df['Ngrams'] = df['text_Tokens'].apply(lambda x: criar_ngrams(x, 2))  # Exemplo de criação de bigrams (n=2)

# Contar a frequência de cada n-gram
ngrams_freq = Counter([gram for sublist in df['Ngrams'] for gram in sublist])

# Exibir os 10 n-grams mais frequentes
print(ngrams_freq.most_common(10))

In [ ]:
# Extrair as palavras dos n-grams
words = [word for sublist in df['Ngrams'] for gram in sublist for word in gram]

# Contar a frequência das palavras
word_freq = Counter(words)

# Converter o resultado do contador em um DataFrame pandas
df_word_freq = pd.DataFrame.from_dict(word_freq, orient='index').reset_index()
df_word_freq.columns = ['Palavra', 'Frequência']

# Ordenar o DataFrame por frequência
df_word_freq = df_word_freq.sort_values(by='Frequência', ascending=False)
df_word_freq

In [ ]:
# Ordenar o DataFrame por frequência
df_word_freq = df_word_freq.sort_values(by='Frequência', ascending=False)

# Plotar o gráfico de barras com Seaborn
plt.figure(figsize=(10, 5))
sns.barplot(x='Frequência', y='Palavra', data=df_word_freq.head(25), color='skyblue', edgecolor='black')
plt.title('Top 25 Palavras dos N-grams')
plt.xlabel('Frequência')
plt.ylabel('Palavra')
plt.show()

In [ ]:
# Divisão treino e teste
x = df['Texto_limpo']
y = df['Sentimento']

In [ ]:
# Visualizando linhas e colunas x
x.shape

In [ ]:
# Visualizando linhas e colunas y
y.shape

In [ ]:
# Importando bibliotca
from sklearn.feature_extraction.text import TfidfVectorizer

# Vetorização dos textos usando TF-IDF
vectorizer = TfidfVectorizer()

# Treinamento
X = vectorizer.fit_transform(x)

# Visualizando
vectorizer

In [ ]:
# Importando biblioteca
from sklearn.decomposition import PCA

# Instanciando o PCA
# Redução de dimensionalidade com PCA para visualização
pca = PCA(n_components=2)

# Treinamento modelo
X_pca = pca.fit_transform(X.toarray())

# Visualizando
pca

In [ ]:
from sklearn.cluster import KMeans

# Calcular a inércia para diferentes números de clusters
inertia = []
for i in range(1, 20):
    kmeans = KMeans(n_clusters=i, random_state=42)
    kmeans.fit(X)
    inertia.append(kmeans.inertia_)

# Plotar a curva de cotovelo (elbow)
plt.plot(range(1, 20), inertia, marker='o')
plt.xlabel('Número de Clusters')
plt.ylabel('Inércia')
plt.title('Método do Cotovelo para Determinar Número de Clusters')
plt.show()

In [ ]:
# Escolher o número de clusters
# Defina o número de clusters com base no método do cotovelo
num_clusters = 5

In [ ]:
# Criar e ajustar o modelo K-means com o número de clusters escolhido
kmeans = KMeans(n_clusters=num_clusters, random_state=42)
clusters = kmeans.fit_predict(X)

In [ ]:
# Adicionar os clusters ao DataFrame
df['Cluster_kemans'] = clusters

In [ ]:
# Visualização dos clusters
# Redução de dimensionalidade com PCA para visualização
pca = PCA(n_components=2)

# Treinamento modelo
X_pca = pca.fit_transform(X.toarray())

# Visualizando
pca

In [ ]:
# Visualização dos clusters
plt.figure(figsize=(10, 6))

scatter = plt.scatter(X_pca[:, 0],
                      X_pca[:, 1],
                      c=clusters)

plt.xlabel('Componente Principal 1')
plt.ylabel('Componente Principal 2')
plt.title('Clusterização dos Textos - Kmeans')

plt.show()

## **Modelo LLM Gemini**

In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

In [ ]:
# Visualizando driver
!nvidia-smi

In [ ]:
# Certifique-se de que a GPU está disponível
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

In [ ]:
# Mover o modelo para o dispositivo correto para GPU
model = model.to(device)
model

In [ ]:
model.eval()

In [ ]:
# Importando bibliotca
from sklearn.feature_extraction.text import TfidfVectorizer

# Vetorização dos textos usando TF-IDF
vectorizer = TfidfVectorizer()

# Treinamento
train = vectorizer.fit_transform(x)

# Visualizando
vectorizer

In [ ]:
# Tokenização dos texto
tokenizer = AutoTokenizer.from_pretrained("describeai/gemini")

In [ ]:
# Modelo LLM Gemini

# Carregar o tokenizador e o modelo
model = AutoModelForSeq2SeqLM.from_pretrained("describeai/gemini")

In [ ]:
# Lista para armazenar os textos gerados
textos_gerados = []
epocas = []

# Número total de épocas
total_epocas = len(df)
print("Número total de épocas:", total_epocas)

In [ ]:
# Loop sobre os primeiros 100 textos limpos
for i, texto_limpo in enumerate(df['Texto_limpo'][:total_epocas], 1):  # Começando em 1 para contar as épocas a partir de 1

    # Tokenizar o texto
    inputs = tokenizer(texto_limpo, return_tensors="pt", max_length=512, truncation=True)
    inputs = {key: tensor.to(device) for key, tensor in inputs.items()}  # Enviar os inputs para o dispositivo correto

    # Passar os inputs pelo modelo
    with torch.no_grad():
        outputs = model.generate(**inputs)  # Removido .to(device)

    # Decodificar as previsões
    texto_gerado = tokenizer.decode(outputs[0], skip_special_tokens=True)

    # Adicionar o texto gerado à lista
    textos_gerados.append(texto_gerado)

    # Adicionar o número da época à lista
    epocas.append(i)

    # Imprimir o texto gerado e o número da época
    print()
    print(f"Época {i} de {total_epocas} - Texto Gerado:", texto_gerado)
    print()

In [ ]:
# Limitar o DataFrame ao número de textos gerados
df = df.head(len(textos_gerados_limitados))

In [ ]:
# Adicionar os textos gerados limitados como uma nova coluna ao DataFrame
df['Texto_gerado_LLM_Gemini'] = textos_gerados_limitados

In [ ]:
df.Texto_gerado_LLM_Gemini.head()

In [ ]:
# 1. Pré-processamento dos dados
textos_limpos = df['Texto_gerado_LLM_Gemini']

In [ ]:
# 2. Representação dos dados

# Ajuste os parâmetros conforme necessário
vectorizer = TfidfVectorizer(max_features=1000)

# Treinamento
x_train = vectorizer.fit_transform(textos_limpos)

# Visualizando
vectorizer

In [ ]:
from sklearn.cluster import KMeans

# Calcular a inércia para diferentes números de clusters
inertia = []
for i in range(1, 11):
    kmeans = KMeans(n_clusters=i, random_state=42)
    kmeans.fit(x_train)
    inertia.append(kmeans.inertia_)

# Plotar a curva de cotovelo (elbow)
plt.plot(range(1, 11), inertia, marker='o')
plt.xlabel('Número de Clusters')
plt.ylabel('Inércia')
plt.title('Método do Cotovelo para Determinar Número de Clusters')
plt.show()

In [ ]:
# Escolher o número de clusters
# Defina o número de clusters com base no método do cotovelo
num_clusters = 5

In [ ]:
# 3. Modelagem de clusterização
kmeans = KMeans(n_clusters=5, random_state=42)  # Defina o número de clusters
kmeans.fit(x_train)

In [ ]:
from sklearn.metrics import silhouette_score

# 4. Avaliação dos clusters
silhouette_avg = silhouette_score(x_train, kmeans.labels_)
print("Silhouette Score:", silhouette_avg)

In [ ]:
# 5. Interpretação dos clusters
df['Cluster_LLM'] = kmeans.labels_

In [ ]:
print(df.groupby('Cluster_LLM').size())

In [ ]:
# Importando biblioteca
from sklearn.manifold import TSNE
from sklearn.decomposition import TruncatedSVD

# Definir a perplexidade desejada
#perplexidade = 35

# Reduzir a dimensionalidade dos dados usando t-SNE
tsne = TSNE(n_components=3, init='random', random_state=42)

# Treinamento modelo
X_tsne = tsne.fit_transform(x_train)

# Reduzir a dimensionalidade dos dados usando TruncatedSVD
svd = TruncatedSVD(n_components=3)

# Treinamento modelo
X_svd = svd.fit_transform(x_train)

# Visualizando modelo
tsne

In [ ]:
# Visualizando modelo
svd

In [ ]:
# Mapear os rótulos de cluster para os sentimentos correspondentes
cluster_sentimentos = {0:
                       'Negativo', 1:
                       'Neutro', 2:
                       'Positivo'}

# Criar subplots
fig, axs = plt.subplots(1, 2, figsize=(15, 6))

# Plotar gráfico de dispersão com os clusters após TruncatedSVD
scatter_svd = sns.scatterplot(x=X_svd[:, 0], y=X_svd[:, 1], hue=kmeans.labels_, palette='bright', alpha=0.5, ax=axs[0])
axs[0].set_title('Clusters TruncatedSVD LLM - Gemini')
axs[0].set_xlabel('Componente Principal 1')
axs[0].set_ylabel('Componente Principal 2')
scatter_svd.legend(title='Cluster LLM', labels=[cluster_sentimentos[i] for i in range(len(cluster_sentimentos))])  # Adicionando legenda

# Plotar gráfico de dispersão com os clusters após t-SNE
scatter_tsne = sns.scatterplot(x=X_tsne[:, 0], y=X_tsne[:, 1], hue=kmeans.labels_, palette='bright', alpha=0.5, ax=axs[1])
axs[1].set_title(f'Clusters t-SNE LLM - Gemini (Perplexidade={perplexidade})')
axs[1].set_xlabel('Componente 1 (t-SNE)')
axs[1].set_ylabel('Componente 2 (t-SNE)')
scatter_tsne.legend(title='Cluster LLM', labels=[cluster_sentimentos[i] for i in range(len(cluster_sentimentos))])  # Adicionando legenda

plt.tight_layout()
plt.show()

In [ ]:
# Salvando dataset
df.to_csv('dataset_final.csv', index=False)

In [ ]:
# Salvando dataset
from transformers import AutoModelForSeq2SeqLM

# Supondo que 'model' é o seu modelo LLM
model.save_pretrained("caminho/para/salvar/o/modelo")

In [ ]:
# Salvando modelo
model.save_pretrained("meu_modelo_gemini")

In [ ]:
# Deploy

In [ ]:
from transformers import AutoModelForSeq2SeqLM

# Carregando modelo
model = AutoModelForSeq2SeqLM.from_pretrained("/content/meu_modelo_gemini")

In [ ]:
# Modelo
model = AutoModelForSeq2SeqLM.from_pretrained("meu_modelo_gemini")

In [ ]:
# Gerar texto
input_text = "A inteligência artificial"
input_ids = tokenizer.encode(input_text, return_tensors="pt")
output = model.generate(input_ids, max_length=100, num_return_sequences=1, temperature=0.9)

# Decodificar e imprimir o texto gerado
generated_text = tokenizer.decode(output[0], skip_special_tokens=True)
print("Texto gerado:", generated_text)